In [ ]:
import sys
import os
import time
import pandas as pd
sys.path.insert(0, os.path.realpath('..'))
import django
django.setup()
from django.db import connections
from firecares.firestation.models import FireDepartment, FireStation, NFIRSStatistic

fdid = '20019'
state = 'NJ'
years = tuple(map(int, pd.read_sql_query('select distinct(extract(year from inc_date)) as year from buildingfires order by year desc;', connections['nfirs'])['year']))

params = dict(fdid=fdid, state=state, years=years)

###  Firefighter casualty segmented by structure hazard level vs flat NFIRS statistics

In [ ]:
%%time

q = """select count(1), extract(year from inc_date) as year, COALESCE(b.risk_category, 'N/A') as risk_category
FROM ffcasualty a left join (SELECT
  *
FROM (
  SELECT state, 
    fdid,
    inc_date,
    inc_no,
    exp_no,
    geom,
    b.parcel_id,
    b.wkb_geometry,
    b.risk_category,
    ROW_NUMBER() OVER (PARTITION BY state, fdid, inc_date, inc_no, exp_no, geom ORDER BY st_distance(st_centroid(b.wkb_geometry), a.geom)) AS r
  FROM (select * from incidentaddress where state=%(state)s and fdid=%(fdid)s) a
     left join parcel_risk_category_local b on a.geom && b.wkb_geometry
     ) x
WHERE x.r = 1) b using (state, inc_date, exp_no, fdid, inc_no) where state=%(state)s and fdid=%(fdid)s and extract(year from inc_date) in %(years)s
GROUP by extract(year from inc_date), COALESCE(b.risk_category, 'N/A')
ORDER BY extract(year from inc_date) DESC"""
    
print pd.read_sql_query(q, connections['nfirs'], params=params)

In [ ]:
%%time
q = """select extract(year from inc_date) as year, count(*) from ffcasualty where fdid=%(fdid)s and state=%(state)s and extract(year from inc_date) in %(years)s group by year order by year desc;"""
cur = connections['nfirs'].cursor()

cur.


print pd.read_sql_query(q, connections['nfirs'], params=params)

### Building fires segmented by structure hazard level vs existing NFIRS statistics

In [ ]:
%%time

q = """select count(1), extract(year from alarm) as year, COALESCE(b.risk_category, 'N/A') as risk_category
FROM buildingfires a left join (SELECT
  *
FROM (
  SELECT state,
    fdid,
    inc_date,
    inc_no,
    exp_no,
    geom,
    b.parcel_id,
    b.wkb_geometry,
    b.risk_category,
    ROW_NUMBER() OVER (PARTITION BY state, fdid, inc_date, inc_no, exp_no, geom ORDER BY st_distance(st_centroid(b.wkb_geometry), a.geom)) AS r
  FROM (select * from incidentaddress where state=%(state)s and fdid=%(fdid)s) a
     left join parcel_risk_category_local b on a.geom && b.wkb_geometry
     ) x
WHERE x.r = 1) b using (state, inc_date, exp_no, fdid, inc_no) where state=%(state)s and fdid=%(fdid)s and extract(year from alarm) in %(years)s
GROUP by extract(year from alarm), COALESCE(b.risk_category, 'N/A')
ORDER BY extract(year from alarm) DESC"""

print pd.read_sql_query(q, connections['nfirs'], params=params)

In [ ]:
%%time

q = """select extract(year from inc_date) as year, count(*) from buildingfires where fdid=%(fdid)s and state=%(state)s and extract(year from inc_date) in %(years)s group by year order by year desc;"""
    
print pd.read_sql_query(q, connections['nfirs'], params=params)

### Civilian casualities segmented by structure hazard level vs existing NFIRS statistics

In [ ]:
%%time

q = """select count(1), extract(year from inc_date) as year, COALESCE(b.risk_category, 'N/A') as risk_category
FROM civiliancasualty a left join (SELECT
  *
FROM (
  SELECT state, 
    fdid,
    inc_date,
    inc_no,
    exp_no,
    geom,
    b.parcel_id,
    b.wkb_geometry,
    b.risk_category,
    ROW_NUMBER() OVER (PARTITION BY state, fdid, inc_date, inc_no, exp_no, geom ORDER BY st_distance(st_centroid(b.wkb_geometry), a.geom)) AS r
  FROM (select * from incidentaddress where state=%(state)s and fdid=%(fdid)s) a
     left join parcel_risk_category_local b on a.geom && b.wkb_geometry
     ) x
WHERE x.r = 1) b using (state, inc_date, exp_no, fdid, inc_no) where state=%(state)s and fdid=%(fdid)s and extract(year from inc_date) in %(years)s
GROUP by extract(year from inc_date), COALESCE(b.risk_category, 'N/A')
ORDER BY extract(year from inc_date) DESC"""

print pd.read_sql_query(q, connections['nfirs'], params=params)

In [ ]:
%%time

q = """select extract(year from inc_date) as year, count(*) from civiliancasualty where fdid=%(fdid)s and state=%(state)s and extract(year from inc_date) in %(years)s group by year order by year desc;"""

print pd.read_sql_query(q, connections['nfirs'], params=params)